# File submission workflow for Dagster
Many data engineering (using Dagster/Airflow/etc) require a method to manually submit and trigger ingestion of data.
This notebook, together with the associated sample Dagster defs file and utility package, demonstrates one way of implementing such a method.

## Trying it out
1. run `uv run dagster dev -f file_submission_defs.py` in a terminal to start the sample Dagster project
2. open this notebook after first running `uv run --with jupyter --with jupyterlab_vim jupyter lab` in another terminal

## Inspect utilities

In [1]:
import pandas as pd
from util import submit_film_file

submit_film_file?

Signature:
submit_film_file(
    df: pandera.typing.pandas.DataFrame[util.FilmSchema],
    filename: str,
)
Docstring: Submit film dataframe to dagster, if it passes schema validation.
File:      ~/dkstat/dagster-recipes/file-submission/util.py
Type:      function

In [2]:
from util import FilmSchema

FilmSchema??

Init signature: FilmSchema(*args, **kwargs) -> pandera.typing.common.DataFrameBase[~TDataFrameModel]
Docstring:     
Model of a pandas :class:`~pandera.api.pandas.container.DataFrameSchema`.

*new in 0.5.0*

See the :ref:`User Guide <dataframe-models>` for more.
Source:        
class FilmSchema(pa.DataFrameModel):
    name: Series[str]
    lead_actor: Series[str]
    rating: Series[int] = pa.Field(ge=0, le=10)
File:           ~/dkstat/dagster-recipes/file-submission/util.py
Type:           MetaModel
Subclasses:     

## Try to submit some files
Note that blabla TODO instant feedback is valuable

In [3]:
try:
    submit_film_file(
        df=pd.DataFrame(
            [
                {"name": "Rogue One: A Star Wars Story", "lead_actorr": "Mads Mikkelsen", "rating": 7},
                {"name": "Bastarden", "lead_actor": "Mads Mikkelsen", "rating": 7.7},
                {"name": "Druk", "lead_actor": "Mads Mikkelsen", "rating": 11},
            ]
        ),
        filename="best_mikkelsen_films.csv",
    )
except Exception as e:
    print(type(e).__name__ + ":")
    print(e)

SchemaError:
non-nullable series 'lead_actor' contains null values:
0    NaN
Name: lead_actor, dtype: object


In [4]:
try:
    submit_film_file(
        df=pd.DataFrame(
            [
                {"name": "Rogue One: A Star Wars Story", "lead_actor": "Mads Mikkelsen", "rating": 7},
                {"name": "Bastarden", "lead_actor": "Mads Mikkelsen", "rating": 7.7},
                {"name": "Druk", "lead_actor": "Mads Mikkelsen", "rating": 11},
            ]
        ),
        filename="best_mikkelsen_films.csv",
    )
except Exception as e:
    print(type(e).__name__ + ":")
    print(e)

SchemaError:
expected series 'rating' to have type int64, got float64


In [5]:
try:
    submit_film_file(
        df=pd.DataFrame(
            [
                {"name": "Rogue One: A Star Wars Story", "lead_actor": "Mads Mikkelsen", "rating": 7},
                {"name": "Bastarden", "lead_actor": "Mads Mikkelsen", "rating": 8},
                {"name": "Druk", "lead_actor": "Mads Mikkelsen", "rating": 11},
            ]
        ),
        filename="best_mikkelsen_films.csv",
    )
except Exception as e:
    print(type(e).__name__ + ":")
    print(e)

SchemaError:
Column 'rating' failed element-wise validator number 1: less_than_or_equal_to(10) failure cases: 11


In [6]:
try:
    submit_film_file(
        df=pd.DataFrame(
            [
                {"name": "Rogue One: A Star Wars Story", "lead_actor": "Mads Mikkelsen", "rating": 7},
                {"name": "Bastarden", "lead_actor": "Mads Mikkelsen", "rating": 8},
                {"name": "Druk", "lead_actor": "Mads Mikkelsen", "rating": 10},
            ]
        ),
        filename="best_mikkelsen_films.csv",
    )
except Exception as e:
    print(type(e).__name__ + ":")
    print(e)

![](materialize_success.png)